In [1]:
import os
import configparser

import pandas as pd

import psycopg2

import time

In [2]:
# 쿼리 조회 시간을 체크
def my_timer(original_function):

    def wrapper(*args, **kwargs):
        t1 = time.time()
        result = original_function(*args, **kwargs)
        t2 = time.time() - t1
        print('{} 함수가 실행된 총 시간: {} 초'.format(original_function.__name__, t2))
        return result

    return wrapper

@my_timer
# DB에서 데이터를 불러오기 위함
def query_select(query, params = []):

    root_path = os.path.dirname(os.getcwd())
    
    config  = configparser.ConfigParser()
    config.read(root_path + '/source/db_config.ini', encoding = 'utf-8')

    USERNAME = config['POSTGRES']['USERNAME']
    PASSWORD = config['POSTGRES']['PASSWORD']
    HOST = config['POSTGRES']['HOST']
    PORT = config['POSTGRES']['PORT']
    DATABASE = config['POSTGRES']['DATABASE']
    
    conn = psycopg2.connect(host=HOST,
                            user=USERNAME,
                            password=PASSWORD,
                            port=PORT,
                            database=DATABASE
                           )
    
    with conn.cursor() as cursor:
        
        if len(params) > 0:
            cursor.execute(query, params)
        else:
            cursor.execute(query)
            
        headers = [x[0] for x in cursor.description]
        data = cursor.fetchall()
    
    conn.close()
       
    return pd.DataFrame(columns=headers, data=data)

In [3]:
# 같은 날 처방받은 것을 합쳐준다.
qry = """
    with tmp_concept_table(concept_id, concept_name) as (
        values
        (19018935, 'digoxin'),
        (1539411, 'simvastatin'),
        (1539463, 'simvastatin'),
        (19075601, 'clopidogrel'),
        (1115171, 'naproxen')
    )

            select 
                distinct 
                person_id,
                drug_exposure_start_date,
                STRING_AGG(distinct concept_name, ',' order by concept_name) concept_list
            from 
                drug_exposure a
                inner join tmp_concept_table b 
                on drug_concept_id = b.concept_id
            group by
                person_id,
                drug_exposure_start_date
            order by 
                person_id,
                drug_exposure_start_date
    ;
"""

In [4]:
result = query_select(qry)
result

query_select 함수가 실행된 총 시간: 0.23401403427124023 초


,person_id,drug_exposure_start_date,concept_list
0,2955,2016-07-24,naproxen
1,12111,1995-08-06,naproxen
2,13576,1958-03-13,naproxen
3,13576,2017-12-06,digoxin
4,13576,2018-12-12,digoxin
...,...,...,...
3570,2833968,2019-09-09,"clopidogrel,digoxin,simvastatin"
3571,2833968,2019-10-14,"clopidogrel,digoxin,simvastatin"
3572,2833968,2019-11-18,"clopidogrel,digoxin,simvastatin"
3573,2833968,2019-12-09,"clopidogrel,digoxin,simvastatin"


In [5]:
person_id = ''
concept_list = ''

ids = set(result.person_id)
data_list =  []

# 패턴 내역을 담을 리스트를 만든다.
for idx, person in enumerate(ids):
    data_list.append(
        { 
            'person' : person,
            'concept_list' : []
        }
    )
    
# 위에서 생성된 패턴내역 리스트에 데이터를 넣는다.
# 1. 바로 전 값과 같은 값이면, 패턴 변화가 없다고 판단하여 리스트에 담지 않는다.
# 2. 바로 전 값과 다른 값이면,
#     a. 패턴에 변화가 있거나
#     b. 다른 사람으로 변경된 것이기 때문에 리스트에 값을 담는다.
for idx, data in enumerate(ids):
    tmp_df = result[result.person_id == data]
    
    tmp_person = ''
    tmp_concept = ''
    tmp_list = []
    for values in tmp_df.values:
        
        if (values[0] == tmp_person) & (values[2] == tmp_concept):
            tmp_list.append(values[2])
            
        else:
            tmp_person = values[0]
            tmp_concept = values[2]
            
            data_list[idx]['concept_list'].append(values[2])

In [6]:
# 위에서 생성된 패턴리스트를 ' -> '로 묶는다.
# 패턴이 몇번 변화했는지에 대한 값도 넣어준다.
result_df = pd.DataFrame(data_list)
result_df['result_pattern'] = result_df.concept_list.apply(lambda x : ' -> '.join(x))
result_df['pattern_cnt'] = result_df.concept_list.apply(lambda x : len(x) - 1)

In [7]:
result_df.sort_values('pattern_cnt', ascending=False)

,person,concept_list,result_pattern,pattern_cnt
59,2742641,"[simvastatin, clopidogrel,simvastatin, simvast...","simvastatin -> clopidogrel,simvastatin -> simv...",47
102,737925,"[simvastatin, clopidogrel,simvastatin, simvast...","simvastatin -> clopidogrel,simvastatin -> simv...",36
373,1476603,"[naproxen, clopidogrel,simvastatin, simvastati...","naproxen -> clopidogrel,simvastatin -> simvast...",27
197,832694,"[naproxen, simvastatin, digoxin, simvastatin, ...",naproxen -> simvastatin -> digoxin -> simvasta...,27
115,1446601,"[simvastatin, clopidogrel,simvastatin, simvast...","simvastatin -> clopidogrel,simvastatin -> simv...",26
...,...,...,...,...
144,715658,[naproxen],naproxen,0
143,586627,[naproxen],naproxen,0
142,1985403,[simvastatin],simvastatin,0
141,420731,[naproxen],naproxen,0


In [8]:
# 제시된 패턴과 같은 패턴이 있는지 찾는다.
# 결과 : 같은 패턴이 없음...
search_pattern = 'digoxin -> naproxen -> clopidogre'
result_df[result_df.result_pattern.apply(lambda x : 1 if search_pattern in x else 0) == 1]

,person,concept_list,result_pattern,pattern_cnt
